In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import scipy.stats as stats

In [3]:
wine_data = "Datasets/winemag-data-130k-v2.csv"

In [4]:
wine_data_df = pd.read_csv(wine_data)

In [5]:
wine_data_df.head(2)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


In [6]:
wine_data_df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [7]:
reduced_wine_data_df = wine_data_df[["country", "points", "price", "variety", "province", "winery"]]

In [8]:
drop_na_df = reduced_wine_data_df.dropna()

In [9]:
drop_na_df.count()

country     120915
points      120915
price       120915
variety     120915
province    120915
winery      120915
dtype: int64

In [10]:
#groupbys for dataset
country_group = drop_na_df.groupby("country")
province_group = drop_na_df.groupby("province")
winery_group = drop_na_df.groupby("winery")
variety_group = drop_na_df.groupby("variety")
country_by_province = drop_na_df.groupby(["country", "province"])
country_by_province.mean()
country_by_variety = drop_na_df.groupby(["country", "variety"])
country_group.mean()

,points,price
country,,
Argentina,86.710330,24.510117
Armenia,87.500000,14.500000
Australia,88.595466,35.437663
Austria,90.190782,30.762772
Bosnia and Herzegovina,86.500000,12.500000
Brazil,84.659574,23.765957
Bulgaria,87.936170,14.645390
Canada,89.377953,35.712598
Chile,86.495130,20.787316


In [37]:

most_produced_varieties= drop_na_df.groupby("variety").count()["points"].sort_values(ascending = False)
most_produced_varieties_df = pd.DataFrame({"Number of Bottles":most_produced_varieties})
top_10_varieties = most_produced_varieties_df.iloc[0:10, :].index.to_list()
top_10_varieties

['Pinot Noir',
 'Chardonnay',
 'Cabernet Sauvignon',
 'Red Blend',
 'Bordeaux-style Red Blend',
 'Riesling',
 'Sauvignon Blanc',
 'Syrah',
 'Rosé',
 'Merlot']

In [12]:
country_production = drop_na_df["country"].value_counts()

In [13]:
#Number of bottles with score greater than 80 indexed by country in descending order
sort_production_df = pd.DataFrame({"Bottles with Score >80":country_production})
#iloc df of top 10 producers for graphing
top_10_producers = sort_production_df.iloc[0:10,:]
top_10_producers

,Bottles with Score >80
US,54265
France,17776
Italy,16914
Spain,6573
Portugal,4875
Chile,4415
Argentina,3756
Austria,2799
Australia,2294
Germany,2120


In [41]:
#average score in descending order
avg_points_descending=country_group.mean()["points"].sort_values(ascending=False)
sort_scores = pd.DataFrame({"Average Score":avg_points_descending})
#iloc of top 10 scorers for graphing (convert to value_counts for series if needed)
top_10_scorers_df = sort_scores.iloc[0:10,:]
#list for loc searching of dataframes
top_10_scorers_list = top_10_scorers_df.index.to_list()


,Average Score
country,
England,91.550725
India,90.222222
Austria,90.190782
Germany,89.836321
Canada,89.377953
Hungary,89.165517
China,89.000000
France,88.734867
Luxembourg,88.666667


In [15]:
#Dataframe of top 10 producers average score (optional sort by descending)
top_10_producers_list = top_10_producers.index.to_list()
top_10_producers_scores = sort_scores.loc[top_10_producers_list,:]
top_10_producers_scores.sort_values(by = "Average Score", ascending = False)





,Average Score
country,
Austria,90.190782
Germany,89.836321
France,88.734867
Italy,88.618186
Australia,88.595466
US,88.566387
Portugal,88.316718
Spain,87.290735
Argentina,86.710330


In [26]:
#highest scoring varieties
mean_by_variety = variety_group.mean()
top_varieties = mean_by_variety.sort_values(by=["points"], ascending = False)
#most reviewed varieties
most_pop_varieties = variety_group.count().sort_values(by = ['points'], ascending = False)
top_10_varieties = most_pop_varieties.iloc[0:10,:].index.to_list()
#top 10 number of reviews by score and price
top_10_varieties_df = mean_by_variety.loc[top_10_varieties, :]
top_10_varieties_df.sort_values(by = ['points'], ascending = False)
top_10_producers

,Bottles with Score >80
US,54265
France,17776
Italy,16914
Spain,6573
Portugal,4875
Chile,4415
Argentina,3756
Austria,2799
Australia,2294
Germany,2120


In [17]:
#Dataframes of top 10 varieties 
riesling_df = drop_na_df[drop_na_df['variety']=="Riesling"]
pinot_noir_df = drop_na_df[drop_na_df['variety']=="Pinot Noir"]
syrah_df = drop_na_df[drop_na_df['variety']=="Syrah"]
bordeaux_df = drop_na_df[drop_na_df['variety']=="Bordeaux-style Red Blend"]
cabernet_sauvignon_df = drop_na_df[drop_na_df['variety']=="Cabernet Sauvignon"]
red_blend_df = drop_na_df[drop_na_df['variety']=="Red Blend"]
chardonnay_df = drop_na_df[drop_na_df['variety']=="Chardonnay"]
sauvignon_blanc_df = drop_na_df[drop_na_df['variety']=="Sauvignon Blanc"]
rose_df = drop_na_df[drop_na_df['variety']=="Rosé"]
merlot_df = drop_na_df[drop_na_df['variety']=="Merlot"]
#I can set the indexes by country for these if we want to run some ttests,
#just be warned i was told that this was bad practice

In [18]:
#Mean groupbys of top 10 varieties by country
#These are means, not counts, and are necessarily means of score *and* price by country for varieties of wine
#You can get these values by grouping by country and variety, but these are simplified dataframes for easy graphing

riesling_mean = riesling_df.groupby("country").mean()
pinot_noir_mean = pinot_noir_df.groupby("country").mean()
syrah_mean = syrah_df.groupby("country").mean()
bordeaux_mean = bordeaux_df.groupby("country").mean()
cabernet_mean = cabernet_sauvignon_df.groupby("country").mean()
red_blend_mean = red_blend_df.groupby("country").mean()
chardonnay_mean = chardonnay_df.groupby("country").mean()
sauvignon_mean = sauvignon_blanc_df.groupby("country").mean()
rose_mean = rose_df.groupby("country").mean()
merlot_mean = merlot_df.groupby("country").mean()

In [53]:
pinot_noir_mean.loc[top_10_producers_list, :]

,points,price
country,,
US,89.610125,44.590646
France,89.768401,81.852157
Italy,86.833333,24.500000
Spain,87.000000,19.818182
Portugal,88.200000,31.933333
Chile,85.921788,19.103352
Argentina,85.611650,22.980583
Austria,90.433735,37.036145
Australia,87.354331,32.283465


In [75]:
us_df = drop_na_df[drop_na_df["country"]=="US"]
france_df = drop_na_df[drop_na_df["country"]=="France"]
italy_df = drop_na_df[drop_na_df["country"]=="Italy"]
spain_df = drop_na_df[drop_na_df["country"]=="Spain"]
portugal_df = drop_na_df[drop_na_df["country"]=="Portugal"]
chile_df = drop_na_df[drop_na_df["country"]=="Chile"]
argentina_df = drop_na_df[drop_na_df["country"]=="Argentina"]
austria_df = drop_na_df[drop_na_df["country"]=="Austria"]
australia_df = drop_na_df[drop_na_df["country"]=="Australia"]
germany_df = drop_na_df[drop_na_df["country"]=="Germany"]

In [88]:
select_countries = ['US','France', 'Italy', 'Spain', 'Portugal', 'Chile', 'Argentina', 'Austria', 'Australia', 'Germany']
select_df = drop_na_df[drop_na_df['country'].isin(select_countries)]

select_df.to_csv("Dataset/top_10_countries.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Dataset/top_10_countries.csv'

In [52]:
top_10_producers_scores

,Average Score
country,
US,88.566387
France,88.734867
Italy,88.618186
Spain,87.290735
Portugal,88.316718
Chile,86.495130
Argentina,86.710330
Austria,90.190782
Australia,88.595466
